In [1]:
import sys
from pathlib import Path

In [2]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))


In [ ]:
from api.events.models import EventModel
from api.db.session import engine


from sqlmodel import select,Session

with Session(engine)  as session:
    query  = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds:True"})

In [ ]:
from api.events.models import EventModel
from sqlalchemy import func
from api.db.session import engine
from timescaledb.hyperfunctions import time_bucket

from sqlmodel import select,Session

with Session(engine)  as session:
    bucket = time_bucket("1 day",EventModel.time)
    query  = (
        select(bucket,EventModel.page,func.count())
        .group_by(bucket,EventModel.page)
        .order_by(
            bucket,
            EventModel.page
        )
        )
    compiled_query = query.compile(compile_kwargs={"literal_binds:True"})
    results = session.exec(query).fetchall()
    

In [6]:
from faker import Faker
import random 
import requests
events = 10_000
pages = ["/","/about","/pricing","/contact","/blog","/products","/login","/signup","/dashboard","/settings"]
fake = Faker()
session_ids = [fake.uuid4() for _ in range(20)]
path  ="/api/events"
base_url = "http://localhost:8002"
create_endpoint  = f"{base_url}{path}"
referrers=[
    "https://google.com",
    "https://facebook.com",
    "https://twitter.com",
    "https://linkedin.com",
    "", 
    "https://github.com"
]
for i in range(events):
    page = random.choice(pages)
    user_agent = random.choice([
        fake.chrome,
        fake.firefox,
        fake.safari,
        fake.opera,
        fake.user_agent
    ])()
    payload = dict(
        page  = random.choice(pages),
        user_agent=user_agent,
        ip_address=fake.ipv4(),
        referrer= random.choice(referrers),
        session_id=random.choice(session_ids),
        duration=random.randint(1,300)
        )
    response = requests.post(create_endpoint,json=payload)
    if response.ok:
        data = response.json()
        print(data)
    else:
        print(response.data)

{'ip_address': '192.160.43.229', 'time': '2025-04-08T23:51:23.363136Z', 'page': '/products', 'session_id': '45c1c3d1-bd3d-4637-865b-c77f323ce422', 'user_agent': 'Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3 like Mac OS X; da-DK) AppleWebKit/531.33.4 (KHTML, like Gecko) Version/4.0.5 Mobile/8B113 Safari/6531.33.4', 'id': 1, 'referrer': '', 'duration': 75}
{'ip_address': '201.231.65.200', 'time': '2025-04-08T23:51:23.447733Z', 'page': '/blog', 'session_id': '971d67f0-2fad-417a-8ef7-158fe852984d', 'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 15_8_2 like Mac OS X) AppleWebKit/534.2 (KHTML, like Gecko) CriOS/61.0.840.0 Mobile/00Z865 Safari/534.2', 'id': 2, 'referrer': 'https://github.com', 'duration': 194}
{'ip_address': '218.77.92.96', 'time': '2025-04-08T23:51:23.500538Z', 'page': '/login', 'session_id': '3da89d6b-b985-4c09-8e21-f3fca1bdf72a', 'user_agent': 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.1 (KHTML, like Gecko) Chrome/32.0.813.0 Safari/534.1', 'id': 3, 'referrer': 'https://g

KeyboardInterrupt: 